In [ ]:
import numpy as np
import soundfile as sf
import scipy
import matplotlib.pyplot as plt
import os
import random
import sklearn
import librosa
import librosa.display

from scipy.io import wavfile
from scipy import signal
from scipy.fft import fft, fftfreq
from sklearn.decomposition import FastICA

plt.rcParams["figure.figsize"] = (15,2)

In [ ]:
dir_animal = "Data Sets/Bird Song/"
dir_music  = "Data Sets/GTZAN/"
dir_speech = "Data Sets/TIMIT/"
dir_envir  = "Data Sets/TUT Acoustic Scenes"
dir_synths = "Data Sets/Synthesizers"

In [ ]:
dir_chosen = dir_speech

# For music, bird song, and acoustic: Fs = 22050, 44100, and 44100
# For speech: Fs = 16000
num_samples = 800
num_sounds  = 256 #len(os.listdir(dir_chosen)) 

X = np.zeros((num_sounds, num_samples))

l = []

while (len(l) < num_sounds):
    temp = random.randrange(0, len(os.listdir(dir_chosen)), 1)
    
    if (temp in l):
        continue
    
    else:
        l.append(temp)

index = 0
for idx, file in enumerate(os.listdir(dir_chosen)):
    filename = os.fsdecode(file)
    
    if (idx not in l):
        continue
    
    if filename.endswith(".wav") or filename.endswith(".WAV"): 
        Fs, data = wavfile.read(dir_chosen + "/" + filename)
                                            
        if (len(data.shape) > 1):
            data = (data[:, 0] + data[:, 1]) / 2
        
        start = random.randrange(0, len(data) - num_samples, 1)
        
        hp_filt  = signal.butter(16, Fs / num_samples, btype='highpass', output='sos', fs=Fs)
        filtdata = signal.sosfilt(hp_filt, data)
        
        data = data[start:start + num_samples]
        
        if (index == 0):
            plt.plot(data)
            plt.title("Original Signal Segement")
            plt.show()
        
        if (index == 0):
            plt.plot(filtdata[start:start + num_samples])
            plt.title("Signal After High-Pass Filter")
            plt.show()
        
        X[index, :] = filtdata[start:start + num_samples]
        
        index = index + 1
        
    else:
        continue
        
print("All signals high-pass filtered at frequency {}. {} samples of length {} randomly"
      " selected for study.".format(Fs / num_samples, num_sounds, num_samples))

print(X.shape)

In [ ]:
def ica(x, samplerate, sources):
    
    tolerance = 1e-20
    iterations = 2000
    
    # Data points
    n = np.size(x, 1)

    # Initialize and normalize W
    w = np.random.rand(sources, np.size(x, 0))
    w = w/np.sqrt(np.sum(np.power(w, 2), axis=1, keepdims=True))

    # Centering and whitening
    print('Whitening data..')
    
    # Rows
    x_mean = x.mean(1, keepdims=True)
    x_centered = x-x_mean
    
    # Whitening
    r_cov = np.cov(x_centered)
    u_cov, s_cov, v_cov = np.linalg.svd(r_cov, hermitian=True)
    v = u_cov @ (np.diag(1/np.sqrt(s_cov)) @ u_cov.T)
    z = v @ x_centered

    # ICA loop
    print('ICA loop..')
    
    for i in range(iterations):
        # Previous W
        wlast = w.copy()
        
        # Current source estimate
        y = w @ z

        # Gradient function
        g = y*np.exp(-0.5*np.power(y, 2))
        dg = (1-np.power(y, 2))*np.exp(-0.5*np.power(y, 2))

        # Update W
        w = (g @ z.transpose())/n - dg.mean(axis=1, keepdims=True)*w
        
        # Normalize W
        w = w/np.sqrt(np.sum(np.power(w, 2), axis=1, keepdims=True))
        
        # Decorrelate W
        u_w, s_w, v_w = np.linalg.svd(w)
        w = u_w @ np.diag(1/s_w) @ u_w.T @ w

        # Check convergence
        delta = np.max(1-np.abs(np.sum(w*wlast, axis=1)))
        
        if np.mod(i, 50) == 0:
            print(i, '=>', delta)
            
        if delta < tolerance:
            y = w @ z
            print(i, '=>', delta)
            print('Converged.')
            return y, w, v, x_mean

    # Output
    y = w @ z
    print('Did not converge.')
    return y, w, v, x_mean


# Call ica function
ICs, w, v, mu = ica(X, Fs, sources=128)

In [ ]:
print(ICs.shape)

In [ ]:
plt.rcParams["figure.figsize"] = (15,2)

filename = "Results/Synthesizers/ICs/IC_"

for x in range(128):
    plt.plot(ICs[x, :])
    plt.xticks([])
    plt.yticks([])
    #plt.savefig(filename + str(x), dpi=200)
    plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (15,2)

filename = "Results/Synthesizers/Specs/Spec_"

centers = []
downs   = []
ups     = []

for idx in range(128):

    f, t, Sxx = signal.spectrogram(ICs[idx, :], Fs, nperseg=16)
    plt.pcolormesh(t, f, Sxx, shading='gouraud')
    plt.ylabel('Frequency (Hz)')
    plt.xlabel('Time (s)')
    #plt.savefig(filename + str(idx), dpi=200)
    plt.show()

    ps = np.square(np.abs(np.fft.fft(ICs[idx, :])))[0:128]
    
    center_flag = False
    down_flag   = False
    up_flag     = False
    
    summ = 0

    for x in range(len(ps)):
        temp = ps[x]
        summ += temp

        if (summ > ps.sum() / 2 and center_flag == False):
            center = x * Fs / (len(ps) * 2)
            center_flag = True

        if (summ > ps.sum() / 4 and down_flag == False):
            down = x * Fs / (len(ps) * 2)
            down_flag = True

        if (summ > ps.sum() * 3 / 4 and up_flag == False):
            up = x * Fs / (len(ps) * 2)
            up_flag = True
    
    centers.append(center)
    downs.append(down)
    ups.append(up)
    
ups   = np.asarray(ups)
downs = np.asarray(downs)
centers    = np.asarray(centers)
bandwidths = ups - downs

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

plt.plot(centers[:], bandwidths[:], '*')
plt.xlabel('Center Frequency (Hz)')
plt.ylabel('Width (Hz)')
plt.title('Synthesizers Sound Filter Characteristics')
#plt.savefig("Results/Synthesizers/ch_filter", dpi=200)
plt.show()